In [31]:
import pandas as pd
import os
import plotly.express as px
import sklearn
import datetime
from datetime import datetime
from datetime import date
from dateutil.rrule import rrule, DAILY
import time
import re

today = datetime.now()
dates = []
a = date(2022, 2, 28)
b = date(2022, 5, 23)

for dt in rrule(DAILY, dtstart=a, until=b):
  dates.append(str(dt.strftime("%b-%d")))

date1 = today.strftime("%b-%d")


os.chdir("/Users/Philip/Documents/NU Econ PhD/Scraper")
#dates =['Feb-28','Mar-01','Mar-02','Mar-03','Mar-04','Mar-05','Mar-06','Mar-07','Mar-08',
#        'Mar-09', 'Mar-10','Mar-11', 'Mar-12', 'Mar-13', 'Mar-14', 'Mar-15','Mar-16','Mar-17',
#        'Mar-18','Mar-19', 'Mar-20', 'Mar-21',  'Mar-22', 'Mar-23', 'Mar-24', 'Mar-25',
#        'Mar-26','Mar-27','Mar-28', 'Mar-29','Mar-30','Mar-31', 'Apr-01', 'Apr-02',
#        'Apr-03', 'Apr-04', 'Apr-05','Apr-06', 'Apr-07', 'Apr-08', 'Apr-09',
#        'Apr-10', 'Apr-11', 'Apr-12', 'Apr-13', 'Apr-14','Apr-15', 'Apr-16',
#        'Apr-17', 'Apr-18', 'Apr-19', 'Apr-20', 'Apr-21', 'Apr-22',
#        'Apr-23', 'Apr-24', 'Apr-25', 'Apr-26', 'Apr-27', 'Apr-28', 
#        'Apr-29', 'Apr-30', 'May-01', 'May-02', 'May-03', 'May-04',
#        'May-05', 'May-06', 'May-07','May-08', 'May-09', 'May-10', 'May-11',
#        'May-12', 'May-13', 'May-14','May-15', 'May-16','May-17','May-18', 'May-19',
#        'May-20', 'May-21', 'May-22','May-23'
#       ]
# file_path_stub = ('p%s%s%s' % (variable, year, month))
df_final=[]
for i in dates:
    file_path = '%s.csv' %i
    df = pd.read_csv(file_path)
    #df['month'] = pd.to_datetime('%s-%s' % (year, month))
    df_final.append(df)

df = pd.concat(df_final, axis=0)

df.drop(columns=df.columns[0],  axis=1, inplace=True)

df['ID'] = df['ID'].replace({'&s=1':''},regex=True).replace({'number=':''},regex=True).replace({'er=':''},regex=True).replace({'-PRS':''},regex=True)
cols = ['IUI Price', 'IUI ART Price','ICI Price','ICI ART Price','IVF Price']
df[cols] = df[cols].replace({'[/$,]*':''},regex=True).apply(pd.to_numeric, errors='coerce')
date_count = df.groupby(['Date']).count()
#date_count = df.groupby(['Date'])['ID'].count()
df['Date'] = df['Date'].str[0:10]

cols =['Height','Weight','Eye Color','Skin Tone','Hair', 'Blood Type','Highest Degree']
df.Height = df['Height'].str.extract(pat='[0-9\'\"]*\(([0-9]*)')
df.Weight = df['Weight'].str.extract(pat='([0-9]*)')
df['Eye Color'] = df['Eye Color'].astype("string")
df['Skin Tone'] = df['Skin Tone'].astype("string")
df.Hair = df.Hair.astype("string")
df['Blood Type'] = df['Blood Type'].astype("string")
df['Highest Degree']=df['Degree'].str.extract(pat='([A-Za-z]*)')
df.Height = df.Height.astype("int")
df.Weight = df.Weight.astype("int")

#REPLACE VIALS LEFT
df = df.replace(
    {
        'IUI Number': {
            ' Max10 ': '10',
            ' Max9 ': '9',
            ' Max8 ': '8',
            ' Max7 ': '7',
            ' Max6 ': '6',
            ' Max5 ': '5',
            ' Please Call ': '1',
            ' Notify Me ': '0'
        }
    },
    regex=True)

df = df.replace(
    {
        'IUI ART Number': {
            ' EvenMax10 ': '10',
            ' EvenMax8 ': '8',
            ' EvenMax6 ': '6',
            ' EvenMax4 ': '4',
            ' Please Call ': '1',
            ' Notify Me ': '0'
        }
    },
    regex=True)
df = df.replace(
    {
        'IUI ART Number': {
            ' Max10 ': '10',
            ' Max9 ': '9',
            ' Max8 ': '8',
            ' Max7 ': '7',
            ' Max6 ': '6',
            ' Max5 ': '5',
            ' Please Call ': '1',
            ' Notify Me ': '0'
        }
    },
    regex=True)
df= df.replace({'IUI ART Number': {'IUI ART[/$0-9,A-Za-z .>]*12345678910112345678910Add to Cart$': '10', 
                                   'IUI ART[/$0-9,A-Za-z .>]*1234567891123456789Add to Cart$': '9',
                                   'IUI ART[/$0-9,A-Za-z .>]*12345678112345678Add to Cart$': '8',
                                   'IUI ART[/$0-9,A-Za-z .>]*123456711234567Add to Cart$': '7',
                                   'IUI ART[/$0-9,A-Za-z .>]*1234561123456Add to Cart$': '6',
                                   'IUI ART[/$0-9,A-Za-z .>]*12345112345Add to Cart$': '5',
                                   'IUI ART[/$0-9,A-Za-z .>]*Please Call$': '1',   
                                   'IUI ART[/$0-9,A-Za-z .>]*Notify Me$': '0',                                                                        
                                }}, regex=True)

df = df.replace(
    {
        'ICI Number': {
            ' Max10 ': '10',
            ' Max9 ': '9',
            ' Max8 ': '8',
            ' Max7 ': '7',
            ' Max6 ': '6',
            ' Max5 ': '5',
            ' Please Call ': '1',
            ' Notify Me ': '0'
        }
    },
    regex=True)

df = df.replace(
    {
        'ICI ART Number': {
            ' EvenMax10 ': '10',
            ' EvenMax8 ': '8',
            ' EvenMax6 ': '6',
            ' EvenMax4 ': '4',
            ' Please Call ': '1',
            ' Notify Me ': '0'
        }
    },
    regex=True)

df = df.replace(
    {
        'ICI ART Number': {
            ' Max10 ': '10',
            ' Max9 ': '9',
            ' Max8 ': '8',
            ' Max7 ': '7',
            ' Max6 ': '6',
            ' Max5 ': '5',
            ' Please Call ': '1',
            ' Notify Me ': '0'
        }
    },
    regex=True)

df= df.replace({'ICI ART Number': {'ICI ART[/$0-9,A-Za-z .>]*12345678910112345678910Add to Cart$': '10', 
                                   'ICI ART[/$0-9,A-Za-z .>]*1234567891123456789Add to Cart$': '9',
                                   'ICI ART[/$0-9,A-Za-z .>]*12345678112345678Add to Cart$': '8',
                                   'ICI ART[/$0-9,A-Za-z .>]*123456711234567Add to Cart$': '7',
                                   'ICI ART[/$0-9,A-Za-z .>]*1234561123456Add to Cart$': '6',
                                   'ICI ART[/$0-9,A-Za-z .>]*12345112345Add to Cart$': '5',
                                   'ICI ART[/$0-9,A-Za-z .>]*Please Call$': '1',   
                                   'ICI ART[/$0-9,A-Za-z .>]*Notify Me$': '0',                                                                         
                                }}, regex=True)


df = df.replace(
    {
        'IVF Number': {
            ' Max10 ': '10',
            ' Max9 ': '9',
            ' Max8 ': '8',
            ' Max7 ': '7',
            ' Max6 ': '6',
            ' Max5 ': '5',
            ' Please Call ': '1',
            ' Notify Me ': '0'
        }
    },
    regex=True)
cols = ['IUI Number', 'IUI ART Number', 'ICI Number', 'ICI ART Number']

for x in cols:
    df[x] = df[x].astype('float')

df=df.iloc[: , 5:]
df.to_csv('df%s.csv' %date1, index=False)


#df = pd.read_csv('dfMar28.csv')
#data=df.groupby(['Ancestry', 'Donor Category',
              #    'Date'])['IUI Number'].mean().reset_index().rename(
              #        columns={'IUI Number': 'IUI Mean Number'
              #                 })
#df_means = pd.DataFrame(data)

#df.groupby(['Ancestry', 'Donor Category',
                #  'Date'])['IUI ART Number'].mean().reset_index().rename(
                #      columns={'IUI ART Number': 'IUI ART Mean Number'
                 #              })

means = df.groupby(['Ancestry', 'Donor Category',
                  'Date']).agg({'IUI Number': ['mean'], 'IUI ART Number': ['mean'],
                                'ICI Number': ['mean'], 'ICI ART Number': ['mean']}).reset_index()


#means=means.drop([1])
#means=means.reset_index()

cols = ['Ancestry', 'Donor Category', 'Date','IUI Number',
        'IUI ART Number', 'ICI Number','ICI ART Number']
#means.rename(columns=cols, axis= 'columns', inplace=True)
means.columns = cols
means.reset_index(col_level =1)
os.chdir("/Users/Philip/Documents/GitHub/animated-plotly/")
means.to_csv('Group_means%s.csv' %date1,index=False)

os.chdir("/Users/Philip/Documents/NU Econ PhD/Scraper")
df = pd.read_csv('df%s.csv'%date1)
df_num = df[['IUI Number','Date','IUI ART Number','ICI Number','ICI ART Number','ID']]
times = df_num.sort_values(by=['ID','Date'])
os.chdir("/Users/Philip/Documents/GitHub/Cryo_Scraper/")
times.to_csv('Aggregaged_Numbers_bydate%s.csv' %date1)

In [2]:
df['IUI ART Number'].value_counts()

0.0     7006
1.0     4095
10.0    2879
8.0      466
6.0      231
9.0       40
7.0        3
Name: IUI ART Number, dtype: int64

In [9]:
dates = []
today_day = 25
for i in range(today_day-1):
    dates.append('May-%s' %date1)
print(dates)

['May-23', 'May-23', 'May-23', 'May-23', 'May-23', 'May-23', 'May-23', 'May-23', 'May-23', 'May-23', 'May-23', 'May-23', 'May-23', 'May-23', 'May-23', 'May-23', 'May-23', 'May-23', 'May-23', 'May-23', 'May-23', 'May-23', 'May-23', 'May-23']


In [5]:
date_day = today.strftime("%d")

In [6]:
date1

'23'

In [20]:
from datetime import date
from dateutil.rrule import rrule, DAILY
dates = []
a = date(2022, 2, 28)
b = date(2022, 5, 23)

for dt in rrule(DAILY, dtstart=a, until=b):
  dates.append(str(dt.strftime("%m-%d")))

In [21]:
dates

['02-28',
 '03-01',
 '03-02',
 '03-03',
 '03-04',
 '03-05',
 '03-06',
 '03-07',
 '03-08',
 '03-09',
 '03-10',
 '03-11',
 '03-12',
 '03-13',
 '03-14',
 '03-15',
 '03-16',
 '03-17',
 '03-18',
 '03-19',
 '03-20',
 '03-21',
 '03-22',
 '03-23',
 '03-24',
 '03-25',
 '03-26',
 '03-27',
 '03-28',
 '03-29',
 '03-30',
 '03-31',
 '04-01',
 '04-02',
 '04-03',
 '04-04',
 '04-05',
 '04-06',
 '04-07',
 '04-08',
 '04-09',
 '04-10',
 '04-11',
 '04-12',
 '04-13',
 '04-14',
 '04-15',
 '04-16',
 '04-17',
 '04-18',
 '04-19',
 '04-20',
 '04-21',
 '04-22',
 '04-23',
 '04-24',
 '04-25',
 '04-26',
 '04-27',
 '04-28',
 '04-29',
 '04-30',
 '05-01',
 '05-02',
 '05-03',
 '05-04',
 '05-05',
 '05-06',
 '05-07',
 '05-08',
 '05-09',
 '05-10',
 '05-11',
 '05-12',
 '05-13',
 '05-14',
 '05-15',
 '05-16',
 '05-17',
 '05-18',
 '05-19',
 '05-20',
 '05-21',
 '05-22',
 '05-23']

In [27]:
df.iloc[: , 6:]

,Unnamed: 0,IUI,IUI ART,ICI,ICI ART,IVF,Height,Weight,Eye Color,Skin Tone,...,IUI ART Number,ICI Number,ICI ART Number,IVF Number,IUI Price,IUI ART Price,ICI Price,ICI ART Price,IVF Price,Highest Degree
0,0,IUIIntrauterine Insemination Ready / Washed/ 1...,IUI ARTIntrauterine Insemination Ready individ...,ICIIntracervical Insemination Ready / Unwashed...,ICI ARTIntracervical Insemination Ready indivi...,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,172,172,Brown,Medium,...,10.0,10.0,10.0,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,1220.0,1220.0,1125.0,1190.0,960.0,PhD
1,1,IUIIntrauterine Insemination Ready / Washed/ 1...,IUI ARTIntrauterine Insemination Ready individ...,ICIIntracervical Insemination Ready / Unwashed...,ICI ARTIntracervical Insemination Ready indivi...,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,182,213,Brown,Medium Light,...,10.0,10.0,10.0,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,1095.0,1160.0,1005.0,1070.0,790.0,MS
2,2,IUIIntrauterine Insemination Ready / Washed/ 1...,IUI ARTIntrauterine Insemination Ready individ...,ICIIntracervical Insemination Ready / Unwashed...,ICI ARTIntracervical Insemination Ready indivi...,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,175,171,Brown,Light,...,10.0,10.0,10.0,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,1220.0,1220.0,1125.0,1190.0,960.0,PhD
3,3,IUIIntrauterine Insemination Ready / Washed/ 1...,IUI ARTIntrauterine Insemination Ready individ...,ICIIntracervical Insemination Ready / Unwashed...,ICI ARTIntracervical Insemination Ready indivi...,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,177,163,Brown,Light,...,10.0,10.0,10.0,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,1095.0,1160.0,1005.0,1070.0,790.0,MS
4,4,IUIIntrauterine Insemination Ready / Washed/ 1...,IUI ARTIntrauterine Insemination Ready individ...,ICIIntracervical Insemination Ready / Unwashed...,ICI ARTIntracervical Insemination Ready indivi...,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,182,266,Brown,Medium,...,10.0,10.0,10.0,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,1220.0,1220.0,1125.0,1190.0,960.0,BA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25943,216,IUIIntrauterine Insemination Ready / Washed/ 1...,IUI ARTIntrauterine Insemination Ready individ...,ICIIntracervical Insemination Ready / Unwashed...,ICI ARTIntracervical Insemination Ready indivi...,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,182,178,Green,Medium,...,0.0,1.0,0.0,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,1220.0,610.0,1125.0,595.0,960.0,BS
25944,217,IUIIntrauterine Insemination Ready / Washed/ 1...,IUI ARTIntrauterine Insemination Ready individ...,ICIIntracervical Insemination Ready / Unwashed...,ICI ARTIntracervical Insemination Ready indivi...,NaN,185,235,Brown,Light,...,0.0,1.0,1.0,NaN,1220.0,740.0,1125.0,720.0,NaN,DO
25945,218,IUIIntrauterine Insemination Ready / Washed/ 1...,IUI ARTIntrauterine Insemination Ready individ...,ICIIntracervical Insemination Ready / Unwashed...,ICI ARTIntracervical Insemination Ready indivi...,NaN,172,155,Green,Medium,...,1.0,0.0,1.0,NaN,1220.0,740.0,1125.0,720.0,NaN,DC
25946,219,IUI ARTIntrauterine Insemination Ready individ...,IUI ARTIntrauterine Insemination Ready individ...,ICIIntracervical Insemination Ready / Unwashed...,ICI ARTIntracervical Insemination Ready indivi...,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,190,197,Hazel,Medium Light,...,0.0,0.0,1.0,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,NaN,580.0,1005.0,535.0,790.0,BA
